# Preliminary analysis of effects of "no duplicates" in packs and ICRs

***This is a speculative analysis that has not been thoroughly bug-tested! Results are provided without warranty or guarantee of correctness.  It is also not applicable to the live version of MTGA but rather a hypothetical version at some point in the future.***

## TLDR
Under a patient wildcard spending strategy, there's minimal difference between the current system and the proposed one in the scenario of going from a new account to the first several constructed decks, whether or not the spending strategy involves chasing ICRs.  In a longer-form scenario of collecting a larger proportion of the available cards, there is a larger difference, with the degree of difference depending on the extent to which ICRs are utilized.  For a more incremental spending strategy, there is likely to be a slightly larger advantage to duplicate prevention, although this is not shown here.

## Specific assumptions of "no duplicates"
1. Once 4 of a specific printing of a card is achieved, it is removed from the possible outcomes of ICRs of that rarity and packs from that set and rarity. All remaining outcomes are equally probable within a given ICR rarity or pack slot.
2. If there are no remaining outcomes, we revert back to filling up the vault with the current percentages.

# Case 1: Building decks from new collection

In [1]:
using ArenaSim
using Statistics
using Distributions
import Random

### Deck selection
The following decks (mainboard) were taken from [MtGGoldfish](https://www.mtggoldfish.com/metagame/standard)

In [2]:
deckdir = joinpath(@__DIR__, "decks", "2018-09-20");
files = readdir(deckdir);
filter!(fn -> isfile(joinpath(deckdir, fn)), files);
files

12-element Array{String,1}:
 "Deck - Bant Nexus.txt"                   
 "Deck - Black-Green Constrictor.txt"      
 "Deck - Blue-Black Midrange.txt"          
 "Deck - Blue-Green Stompy.txt"            
 "Deck - Esper Control.txt"                
 "Deck - Grixis Midrange.txt"              
 "Deck - Mono-Blue Artifacts.txt"          
 "Deck - Mono-Blue Tempo.txt"              
 "Deck - Mono-Red Flame of Keld.txt"       
 "Deck - R_B Aggro.txt"                    
 "Deck - White-Black Knights.txt"          
 "Deck - White-Blue God Pharaoh's Gift.txt"

Here are the number of rares in each deck:

In [3]:
decks = map( fn -> ArenaSim.deckreader_mtgo_format(joinpath(deckdir, fn); strip_sideboard = true), files);
collect(zip(files, [ sum(x.amount * ( minimum(y->y.rarity, x.prints) == 3 ? 1 : 0) for x in z) for z in deckinfo.(decks) ]))

12-element Array{Tuple{String,Int64},1}:
 ("Deck - Bant Nexus.txt", 21)                   
 ("Deck - Black-Green Constrictor.txt", 26)      
 ("Deck - Blue-Black Midrange.txt", 22)          
 ("Deck - Blue-Green Stompy.txt", 20)            
 ("Deck - Esper Control.txt", 27)                
 ("Deck - Grixis Midrange.txt", 28)              
 ("Deck - Mono-Blue Artifacts.txt", 19)          
 ("Deck - Mono-Blue Tempo.txt", 8)               
 ("Deck - Mono-Red Flame of Keld.txt", 16)       
 ("Deck - R_B Aggro.txt", 28)                    
 ("Deck - White-Black Knights.txt", 9)           
 ("Deck - White-Blue God Pharaoh's Gift.txt", 18)

### Generating ICR rarities for someone who plays Quick Constructed

Assuming the following probabilities for an ICR of a given "guaranteed rarity" to be each of the possible rarities

In [4]:
uncommon_icr_probs = [0, 0.85, 0.10, 0.05];
rare_icr_probs = [0,0,0.85,0.15];

In [5]:
first_icr = Dict( (w,uncommon_icr_probs) for w in 0:7)
second_icr = deepcopy(first_icr)
second_icr[6] = rare_icr_probs
second_icr[7] = rare_icr_probs
third_icr = deepcopy(second_icr)
for i in 4:5
    third_icr[i] = rare_icr_probs
end

With a 50% winrate, the expected rarity of each of the 3 ICRs, arranged from lowest to highest value is

In [6]:
winrate = 0.5
avg_first_icr = sum( pdfnegbin_truncated(k; p = winrate, r = 3, maxwins = 7) .* first_icr[k] for k in 0:7 )
avg_second_icr = sum( pdfnegbin_truncated(k; p = winrate, r = 3, maxwins = 7) .* second_icr[k] for k in 0:7 )
avg_third_icr = sum( pdfnegbin_truncated(k; p = winrate, r = 3, maxwins = 7) .* third_icr[k] for k in 0:7 )

avg_first_icr, avg_second_icr, avg_third_icr

([0.0, 0.85, 0.1, 0.05], [0.0, 0.727148, 0.208398, 0.0644531], [0.0, 0.557813, 0.357812, 0.084375])

It costs this player about 90 gold per run.

In [7]:
   qc_payoff = Dict(
        0 => 100,
        1 => 200,
        2 => 300,
        3 => 400,
        4 => 500,
        5 => 600,
        6 => 800,
        7 => 1000
    )

avg_payoff = sum(pdfnegbin_truncated(k; p = winrate, r = 3, maxwins = 7) * qc_payoff[k] for k in 0:7)

410.15625

Which is about 7.5% of their daily income if their quests average 600g and they get 600 out of the 750 non-quest gold:

In [8]:
(500 - avg_payoff) / (600 + 600)

0.07486979166666667

Consider two ICR scenarios:
1. Do 1 quick constructed run everyday, so that there are approximately 7 sets of 3 ICRs with the above rarity distributions per 10 packs (buy about 1 pack per day and then 3 weekly packs)
2. Don't get any ICRs

In [9]:
icrvec = [[ Categorical(avg_first_icr), Categorical(avg_second_icr), Categorical(avg_third_icr) ] for i in 1:7]
for i in 1:3
    push!(icrvec, Categorical{Float64}[])
end
icrcycle = Cycle(  icrvec )
no_icrs = Cycle( [Categorical{Float64}[]] );

### Deck simulation

Building the decks in a random order but the same order for every scenario:

In [10]:
Random.seed!(1234)
const global pregenerated_permutations = [ Random.randperm(length(decks)) for i in 1:5001 ]
function pregenerated_shuffle(i,x)
    perm = pregenerated_permutations[i]
    deepcopy(x)[perm]
end

pregenerated_shuffle (generic function with 1 method)

Simulate both scenarios, with and without duplicate prevention enabled:

In [11]:
Random.seed!(12346)
result_base = shufflesim(deckinfo.(decks); 
    parameters = SimParameters(prevent_duplicates = false, icrs_per_pack = no_icrs ),
    reps = 100, batchsize = 10, shuffledecks = pregenerated_shuffle);

result_noduplicates =  shufflesim(deckinfo.(decks); 
    parameters = SimParameters(prevent_duplicates = true,  icrs_per_pack = no_icrs  ),
    reps = 100, batchsize = 10, shuffledecks = pregenerated_shuffle);

In [12]:
result_icr =  shufflesim(deckinfo.(decks); 
    parameters = SimParameters(prevent_duplicates = false, icrs_per_pack = icrcycle ),
    reps = 100, batchsize = 10, shuffledecks = pregenerated_shuffle);

result_icr_noduplicates =  shufflesim(deckinfo.(decks); 
    parameters = SimParameters(prevent_duplicates = true, icrs_per_pack = icrcycle ),
    reps = 100, batchsize = 10, shuffledecks = pregenerated_shuffle);

### Case 1 Results
The difference in the number of packs opened to reach each deck between any of the scenarios is minimal, smaller than many of the other uncertainties:

In [13]:
show(
    IOContext(stdout, :limit => false), "text/plain", 
    hcat(
        ["Duplicates, no ICRs"; "No Duplicates, No ICRs"; "Duplicates, ICRs"; "No Duplicates, ICRs"],
    vcat(
    mean(reduce(+, values(result_base.packs_opened)), dims = 1),
    mean(reduce(+, values(result_noduplicates.packs_opened)), dims = 1),
    mean(reduce(+, values(result_icr.packs_opened)), dims = 1),
    mean(reduce(+, values(result_icr_noduplicates.packs_opened)), dims = 1)
    )
        )
)

4×13 Array{Any,2}:
 "Duplicates, no ICRs"     88.475  78.255  64.355  59.536  52.905  47.204  40.57   32.795  31.799  30.753  27.367  23.578
 "No Duplicates, No ICRs"  88.754  77.286  63.743  59.933  53.347  48.299  41.714  34.254  32.872  31.841  26.678  23.178
 "Duplicates, ICRs"        85.912  72.661  59.578  53.595  45.827  39.42   32.946  27.321  26.681  25.121  22.314  18.211
 "No Duplicates, ICRs"     85.463  71.567  59.484  53.521  45.641  41.272  34.38   28.57   26.118  25.465  21.891  18.942

## Case 2: Collecting a large number of rares

### Scenario Setup
1. Own 4x every card besides DOM
2. Own only the DOM cards in the 5 monocolor tutorial decks
3. Count packs to owning all the DOM rares under the above 4 scenarios

In [14]:
domrares = [ [(x["name"],4) for x in filter(z -> z["set"] == :DOM && z["rarity"] == 3, ArenaSim.card_db)] ]

1-element Array{Array{Tuple{String,Int64},1},1}:
 [("Benalish Marshal", 4), ("Daring Archaeologist", 4), ("Evra, Halcyon Witness", 4), ("Fall of the Thran", 4), ("Shalai, Voice of Plenty", 4), ("Teshar, Ancestor's Apostle", 4), ("Urza's Ruinous Blast", 4), ("The Antiquities War", 4), ("Karn's Temporal Sundering", 4), ("The Mirari Conjecture", 4)  …  ("Traxos, Scourge of Kroog", 4), ("Cabal Stronghold", 4), ("Clifftop Retreat", 4), ("Hinterland Harbor", 4), ("Isolated Chapel", 4), ("Sulfur Falls", 4), ("Woodland Cemetery", 4), ("Niambi, Faithful Healer", 4), ("Chandra's Outburst", 4), ("Firesong and Sunspeaker", 4)]

In [15]:
newstarter = deepcopy(ArenaSim.basiccards)
for i in findall(z -> z["set"] != :DOM, ArenaSim.card_db)
    newstarter[i] = 4
end

In [16]:
dom_base = shufflesim(deckinfo.(domrares); reps = 1, batchsize = 1000,
    parameters = SimParameters(icrs_per_pack = no_icrs, starter_cards = newstarter, prevent_duplicates = false,
        bonus_packs = Dict((s,0) for s in ArenaSim.sets)));
dom_noduplicates = shufflesim(deckinfo.(domrares); reps = 1, batchsize = 1000,
    parameters = SimParameters(icrs_per_pack = no_icrs, starter_cards = newstarter, prevent_duplicates = true,
        bonus_packs = Dict((s,0) for s in ArenaSim.sets)));
dom_icr = shufflesim(deckinfo.(domrares); reps = 1, batchsize = 1000,
    parameters = SimParameters(icrs_per_pack = icrcycle, starter_cards = newstarter, prevent_duplicates = false,
        bonus_packs = Dict((s,0) for s in ArenaSim.sets)));
dom_icr_noduplicates = shufflesim(deckinfo.(domrares); reps = 1, batchsize = 1000,
    parameters = SimParameters(icrs_per_pack = icrcycle, starter_cards = newstarter, prevent_duplicates = true,
        bonus_packs = Dict((s,0) for s in ArenaSim.sets)));

### Case 2 Results
Here we see a more substantial boost from duplicate prevention.  A $\approx16\%$ reduction in packs needed in the No ICR case and a $\approx39\%$ reduction in the ICR case (although recall we are spending about $7\%$ of our available gold on the QC entries).  The latter is big enough that we wonder what happens if we invest even more gold into QC.

In [17]:
hcat( ["Duplicates, no ICRs"; "No Duplicates, No ICRs"; "Duplicates, ICRs"; "No Duplicates, ICRs"],
      [ mean(dom_base.total_packs); mean(dom_noduplicates.total_packs);mean(dom_icr.total_packs); mean(dom_icr_noduplicates.total_packs) ])

4×2 Array{Any,2}:
 "Duplicates, no ICRs"     264.357
 "No Duplicates, No ICRs"  221.323
 "Duplicates, ICRs"        244.913
 "No Duplicates, ICRs"     151.177

Two QC runs per day is potentially enough gold bleed that we can't buy a pack every single day anymore.  Suppose we're down to 9 packs per week with 12 QC runs:

In [18]:
doubleicrvec = [[ Categorical(avg_first_icr), Categorical(avg_second_icr), Categorical(avg_third_icr), 
        Categorical(avg_first_icr), Categorical(avg_second_icr), Categorical(avg_third_icr) ] for i in 1:6]
for i in 1:3
    push!(doubleicrvec, Categorical{Float64}[])
end

dom_doubleicr_noduplicates = shufflesim(deckinfo.(domrares); reps = 1, batchsize = 1000,
    parameters = SimParameters(icrs_per_pack = Cycle(doubleicrvec), starter_cards = newstarter, prevent_duplicates = true,
        bonus_packs = Dict((s,0) for s in ArenaSim.sets)));
mean(dom_doubleicr_noduplicates.total_packs)

117.576

Incidentally, this is about how many packs you get under this regimen during the duration of one set, which justifies the plausibility of having all the rares besides the set of interest in the long-run.

In [19]:
9 * 52/4

117.0

## Discussions and Concluding Thoughts

As alluded to at the beginning, the simulation uses a conservative wildcard spending strategy that only spends them when there are enough to complete the targeted set of cards.  Many players probably use a more incremental strategy that increases the chance of receiving duplicates.  In this sense, duplicate prevention is likely a bit more helpful than the results presented here, especially in terms of player perception when they receive a duplicate soon after crafting a card.

The bulk scenario does not mean full set completion.  We would likely be a nontrivial amount of mythics short of a full playset.  If you made it this far thanks for reading! 